In [84]:
import jieba
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [6]:
data_dir = '../train/'
features_dir = '../features/'
opscope = pd.read_csv(data_dir + 'base_info.csv',encoding='gbk')
opscope = pd.DataFrame([opscope.id, opscope.opscope]).T

In [90]:
vocabulary = ''
opscope.opscope = opscope.opscope.apply(lambda x:x.split('（依法须经批准的项目')[0])
for i in opscope.opscope:
    vocabulary += i
for i in set(['，', '。', '：', '；', '（', '）', '、','*', '  ', '(', ')']):
    vocabulary = vocabulary.replace(i, ' ')
vocabulary = set(vocabulary)

In [106]:
def replace(x):
    for i in set(['，', '。', '：', '；', '（', '）', '、','*', '(', ')']):
        x = x.replace(i, '')
    return x

def cancats(x):
    s = ''
    for i in x:
        s += i + ' '
    return s.replace('  ', ' ').replace('  ', ' ')

opscope.opscope = opscope.opscope.apply(lambda x:x.split('（依法须经批准的项目')[0])
corpus = opscope.opscope.apply(replace)
corpus = corpus.apply(lambda x:jieba.lcut(x))
corpus = corpus.apply(cancat)

In [92]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                  ('tfidf', TfidfTransformer())]).fit(corpus)

In [112]:
pipe.transform(list(corpus[0]))

<42x9952 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>